<a href="https://colab.research.google.com/github/nh0875/Desafios-Procesamiento-del-Habla-974/blob/main/Desafio_Chatbot%20y%20embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tony "El chatbot coach" (Chatbot con TFIDF/Similitud del coseno)

Voy a utilizar un chatbot con un motor de busqueda por similitud, ya que mi idea es que el chatbot sirva para responder preguntas comunes sobre que acciones tomar en un partido de basquet orientado a personas que no saben jugarlo o que estan comenzando a hacerlo, que hacer en situaciones especificas como tambien situaciones generales de un partido, voy a ayudarme de Gemini para poder abarcar a la mayoria de posibles preguntas que podrian llegarle al bot, como tambien las respuestas que deberia devolver mi bot, se va a llamar **"Tony"**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
questions = [
    "What is the difference between a personal foul and a technical foul?",
    "What is a three-second violation, and how is it applied in the restricted area?",
    "What is the difference between a block and a charge?",
    "What is a screen, and how is it used in offensive play?",
    "What is traveling, and how is it penalized?",
    "What is double dribbling, and how can it be avoided?",
    "What is the difference between a free throw and a baseline throw-in?",
    "What is a defensive switch, and what are the most common types?",
    "What is a 24-second violation, and how is it applied in the game?",
    "What is the importance of ball rotation in offensive play?",
    "If my team is losing by many points at the end of the game, what should we do?",
    "If I have the ball and am being guarded by a very tall defender, what can I do?",
    "If a teammate takes a three-point shot and misses, what should I do?",
    "If I’m defending a player much faster than me, what strategy can I use?",
    "If I find myself alone under the hoop, what type of shot should I attempt?",
    "How can I improve my shooting?",
    "What can I do to improve my ball handling?",
    "How can I improve my defense?",
    "What is a pick-and-roll, and how can I execute it?",
    "What is a fast break, and how can I participate in it?",
    "How can I improve my teamwork?",
    "What is a press defense, and how can I play against it?",
    "How can I stay motivated to keep practicing basketball?"
]

answers = [
    "A personal foul is committed against a player and usually involves physical contact. A technical foul is given for unsportsmanlike actions or protests against the referees.",
    "An offensive player cannot stay in the opponent's restricted area for more than three seconds without dribbling or attempting a shot.",
    "A block is a legal action where a defender prevents the progress of an attacker without committing a foul. A charge occurs when the defender moves into the attacker’s path, illegally stopping their progress.",
    "A screen is an action where a player positions themselves near a defender to obstruct their path, allowing a teammate to receive a pass unguarded.",
    "Traveling is a violation that occurs when a player takes more than one step with the ball in hand without dribbling.",
    "Double dribbling occurs when a player stops dribbling and then resumes dribbling with the same hand. To avoid it, keep the ball in constant motion or pass it to a teammate.",
    "A free throw is awarded as a penalty for a foul and is taken from the free-throw line. A baseline throw-in happens when the ball goes out of bounds on the baseline, and the opposing team gains possession.",
    "A defensive switch is a change in a team's defensive strategy. The most common types are man-to-man defense and zone defense.",
    "A team has 24 seconds to make a shot once they gain possession of the ball. If time runs out without a shot, it results in a violation, and possession goes to the opposing team.",
    "Ball rotation is essential to keep the defense disorganized, create open shooting opportunities, and prevent a single player from bearing the scoring responsibility.",
    "In this situation, it's important to stay calm and continue playing as a team. You can attempt to force turnovers, take three-point shots to close the gap, and avoid committing unnecessary fouls.",
    "You can try to shoot, pass the ball to an open teammate, or use a fake move to create space and score.",
    "You should be ready for the rebound, as the ball is likely to bounce into the area. If you get it, try to take another shot or pass it to a better-positioned teammate.",
    "Maintain a proper distance, use your arms to prevent easy dribbling, and try to anticipate their movements.",
    "If you're alone under the hoop, it's best to attempt an easy shot, such as a layup or a hook.",
    "To improve your shooting, practice regularly, focus on proper technique, and work on your upper body strength.",
    "Practice dribbling with both hands, at different speeds, and in tight spaces. You can also perform coordination drills with the ball.",
    "To improve your defense, work on agility, strength, and the ability to anticipate your opponent's movements.",
    "A pick-and-roll is a play where a player sets a screen (pick) for a teammate who is dribbling, aiming to create a numerical advantage.",
    "A fast break is a quick attack after a defensive rebound or interception. To participate in a fast break, run quickly towards the opposing basket and look for a pass to finish the play.",
    "Communication, trust in your teammates, and a willingness to share the ball are fundamental for good teamwork.",
    "A press defense is when players spread out over the court instead of guarding specific opponents. To counter a press, move the ball quickly and look for open shots.",
    "Set realistic goals, find training partners, join a league, and celebrate your achievements."
]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(questions)

def responder(consut_):
    consulta_vec = vectorizer.transform([consut_])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    indice_mejor_coincidencia = similitudes.argmax()
    return answers[indice_mejor_coincidencia]


In [ ]:
consulta = str(input("Write your question about Basketball: "))
responder(consulta)

Write your question about Basketball: What is a personal foul?


'A personal foul is committed against a player and usually involves physical contact. A technical foul is given for unsportsmanlike actions or protests against the referees.'

# Categorizador de Reseñas (Chatbot con Embeddings, **Modelo FastText** )

Es un bot el cual clasifica reseñas de teclados basandose en palabras clave para determinar si la reseña es positiva o negativa



In [ ]:
!pip install pymupdf SpaCy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 22.0 MB/s eta 0:00:00


In [ ]:
import os
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Cargar un modelo especializado en análisis de sentimientos
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Función para clasificar opiniones
def clasificar_opinion(opinion):
    # Tokenizar el texto
    inputs = tokenizer(opinion, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Hacer predicción
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediccion = torch.argmax(logits, dim=1).item()

    # Interpretar la predicción
    niveles = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    return niveles[prediccion]

# Cargar opiniones desde un archivo de texto
def cargar_opiniones(archivo):
    with open(archivo, 'r', encoding='utf-8') as f:
        opiniones = [line.strip() for line in f if line.strip()]
    return opiniones

# Clasificar 30 opiniones aleatorias
def clasificar_aleatorias(archivo_opiniones, cantidad=30):
    opiniones = cargar_opiniones(archivo_opiniones)

    # Seleccionar 30 opiniones aleatorias
    if len(opiniones) < cantidad:
        print(f"El archivo contiene menos de {cantidad} opiniones. Se clasificarán todas.")
        muestras = opiniones
    else:
        muestras = random.sample(opiniones, cantidad)

    for i, opinion in enumerate(muestras, start=1):
        clasificacion = clasificar_opinion(opinion)
        print(f"Opinión {i}: {opinion}")
        print(f"Clasificación: {clasificacion}\n")

In [ ]:

# Archivo de opiniones
archivo_opiniones = '/content/Untitled document.txt'  # Cambia la ruta si usas otro archivo

# Ejecutar el chatbot para clasificar opiniones aleatorias
try:
    clasificar_aleatorias(archivo_opiniones)
except Exception as e:
    print(f"Error procesando las opiniones: {e}")


Opinión 1: "It lacks backlighting, which is a major drawback."
Clasificación: Neutral

Opinión 2: "It lacks extra features; it’s very basic and subpar."
Clasificación: Neutral

Opinión 3: "The backlighting is too weak, barely visible."
Clasificación: Negative

Opinión 4: "I don’t like it; the keys are uncomfortable, and the noise is annoying."
Clasificación: Negative

Opinión 5: "Works perfectly with my laptop; highly recommended."
Clasificación: Very Positive

Opinión 6: "Good keyboard, but I wouldn’t recommend it for gamers."
Clasificación: Neutral

Opinión 7: "It’s functional but could improve the key quality."
Clasificación: Neutral

Opinión 8: "It serves its purpose, but I expected more."
Clasificación: Neutral

Opinión 9: "Excellent value for money; I would buy it again."
Clasificación: Very Positive

Opinión 10: "The keys are not accurate; many presses aren’t registered."
Clasificación: Negative

Opinión 11: "The keyboard lacks multimedia functions; it’s very basic."
Clasificaci